In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# this is my file's name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "SNHU1234"
#my py file already signs in without needing credentials from here
shelter = AnimalShelter()

# Rescue Queries 

water_rescue_query = {
    "animal_type": "Dog",
    "breed": {
        "$in": [
            "Labrador Retriever Mix",
            "Chesapeake Bay Retriever",
            "Newfoundland"
        ]
    },
    "sex_upon_outcome": "Intact Female",
    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
}


mountain_rescue_query = {
    "animal_type": "Dog",
    "breed": {
        "$in": [
            "German Shepherd",
            "Alaskan Malamute",
            "Old English Sheepdog",
            "Siberian Husky",
            "Rottweiler"
        ]
    },
    "sex_upon_outcome": "Intact Male",
    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
}

disaster_rescue_query = {
    "animal_type": "Dog",
    "breed": {
        "$in": [
            "Doberman Pinscher",
            "German Shepherd",
            "Golden Retriever",
            "Bloodhound",
            "Rottweiler"
        ]
    },
    "sex_upon_outcome": "Intact Male",
    "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
}


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))


# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True, errors = 'ignore')


## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    
    html.A(
        html.Img(
            src='assets/grazioso_logo.png', 
            style={'height': '100px'}
        )
    ),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Div('Abdulrahman Al-Nachar Dashboard', style = {'fontWeight': 'bold', 'fontSize': '20px', 'marginTop': '10px'}),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',  
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        )
    ),

    html.Br(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),

        page_size=10,
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'minWidth': '100px', 'width': '100px', 'maxWidth': '180px'},
        style_data_conditional=[]
    ),
    html.Br(),
    html.Hr(),
    html.Div(
            id='map-id',
            className='col s12 m6',
            ),
    html.Div(
            id='chart-id',
            className='col s12 m6'
            )

])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    
    if dff.empty or dff.shape[1] < 15:
        return [html.Div("Data error: missing location data.")]
    
    row = index[0] if index else 0
    return [
        dl.Map(
        style = {'width': '1000px', 'height': '500px'},
        center = [30.75, -97.48],
        zoom = 10,
        children = [
            dl.TileLayer(id='base-layer-id'),
            dl.Marker(
                position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                children=[
                    dl.Tooltip(dff.iloc[row, 4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row, 9])
                    ])
                ]
            )
        ]
        )
    ]

@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_table(filter_value):
    if filter_value == 'water':
        query = water_rescue_query
    elif filter_value == 'mountain':
        query = mountain_rescue_query
    elif filter_value == 'disaster':
        query = disaster_rescue_query
    else:
        query = {} 

    dff = pd.DataFrame.from_records(shelter.read(query))
    dff.drop(columns=['_id'], inplace=True, errors='ignore')
    return dff.to_dict('records')

@app.callback(
    Output('chart-id', 'children'),
    Input('datatable-id', 'data')
)
def update_pie(viewData):
    dff = pd.DataFrame.from_dict(viewData)
   
    if dff.empty or 'breed' not in dff.columns:
        return [html.Div("No data to display.")]
   
    # Optional: group rare breeds into 'Other' to prevent clutter
    top_breeds = dff['breed'].value_counts().nlargest(5).index
    dff['breed'] = dff['breed'].where(dff['breed'].isin(top_breeds), other='Other')

    fig = px.pie(
        dff,
        names='breed',
        title='Distribution of Breeds',
        color_discrete_sequence=px.colors.sequential.Reds  # Use red shades!
    )

    fig.update_traces(textinfo='percent+label')

    return [
        dcc.Graph(
            figure=fig,
            style={'width': '50%', 'height': '500px'} 
        )
    ]


    
app.run_server(debug=True, port=8055)

Dash app running on http://127.0.0.1:8055/
